# Exploring Land Surface Temperature

[ESA Land Surface Temperature Climate Change Initiative (LST_cci): Monthly Multisensor Infra-Red (IR) Low Earth Orbit (LEO) land surface temperature (LST) time series level 3 supercollated (L3S) global product (1995-2020), version 2.00](https://catalogue.ceda.ac.uk/uuid/785ef9d3965442669bff899540747e28).


In [ ]:
import typing
import warnings

import geopandas as gpd
import numpy as np
import pandas as pd
import regionmask

# rioxarray is not directly referenced, but its `rio` extension of `xarray` is
import rioxarray
import shapely
import xarray as xr
import xrspatial.zonal
from shapely.errors import ShapelyDeprecationWarning

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

## Obtain Geometries for Cholera Outbreak Regions

First, let's read our outbreak data and select all outbreaks in admin2 regions:


In [ ]:
admin2_outbreaks_df = (
    pd.read_csv("data/outbreak_data.csv", parse_dates=["start_date", "end_date"])
    .assign(
        start_year=lambda df: df.start_date.dt.year,
        start_month=lambda df: df.start_date.dt.month,
        duration_in_months=lambda df: np.ceil(
            (df.end_date - df.start_date) / np.timedelta64(1, "M")
        ).astype(int),
    )
    .query("spatial_scale == 'admin2'")
)

admin2_outbreaks_df

Select the distinct location period IDs, so we can select their geometries from
our shapefile:


In [ ]:
admin2_location_period_id_df = admin2_outbreaks_df[
    ["location_period_id"]
].drop_duplicates()

admin2_location_period_id_df

Read our shapefile:


In [ ]:
location_period_id_gdf = typing.cast(
    gpd.GeoDataFrame,
    gpd.read_file("data/AfricaShapefiles/total_shp_0427.shp").rename(
        columns={"lctn_pr": "location_period_id"}
    ),
)

location_period_id_gdf

Merge our distinct location period IDs with the shapefile to obtain the
geometries for only our distinct admin2 outbreak regions:


In [ ]:
admin2_gdf = typing.cast(
    gpd.GeoDataFrame,
    location_period_id_gdf.merge(
        admin2_location_period_id_df,
        how="inner",
        on="location_period_id",
    ).drop_duplicates("geometry", keep=False),
)

# intersecting_areas = (
#     typing.cast(
#         pd.DataFrame,
#         admin2_gdf.apply(
#             lambda row: pd.Series(
#                 [
#                     area := shapely.area(row.geometry),
#                     max_intersecting_area := admin2_gdf.geometry.drop(row.name)
#                     .intersection(row.geometry)
#                     .apply(shapely.area)
#                     .max(),
#                     100.0 * max_intersecting_area / area,
#                 ],
#                 index=["area", "max_intersecting_area", "percent_intersecting_area"],
#             ),
#             axis=1,
#         ),
#     )
#     .query("percent_intersecting_area > 0")
#     .sort_values("percent_intersecting_area", ascending=True)
# )

# display(intersecting_areas)
# admin2_gdf = admin2_gdf.drop(intersecting_areas.index)

admin2_gdf

In [ ]:
display(admin2_gdf.crs)
admin2_gdf.boundary.plot(linewidth=0.2)

In [ ]:
# If you want to look at the entire continent, and compute zonal stats for it,
# uncomment the lines below, which will reassign admin2_gdf to all admin2
# regions in Africa, instead of only the admin2 regions from our outbreak data.

# import pooch
#
# admin2_gdf = typing.cast(
#     gpd.GeoDataFrame,
#     gpd.read_file(
#         pooch.retrieve(
#             "https://geoportal.icpac.net/geoserver/ows?service=WFS"
#             "&version=1.0.0"
#             "&request=GetFeature"
#             "&typename=geonode%3Aafr_g2014_2013_2"
#             "&outputFormat=json"
#             "&srs=EPSG%3A4326"
#             "&srsName=EPSG%3A4326",
#             None,
#         )
#     ),
# )

# display(admin2_gdf)
# display(admin2_gdf.crs)

# admin2_gdf.boundary.plot()

## Compute Zonal Mean Land Surface Temperatures

Compute zones so we can compute zonal statistics.

**WARNING:** This takes several minutes to compute!


Fetch the Land Surface Temperature (LST) data for a specific date from the
[Centre for Environmental Data Analysis Archive](https://archive.ceda.ac.uk/)
(CEDA Archive), and subset it to Africa's bounding box.


In [ ]:
def global_lst_url(*, year: int, month: int) -> str:
    """Return the URL for the global average Land Surface Temperature data file
    for a specific year and month."""

    # Can check status of CEDA core archives at https://stats.uptimerobot.com/vZPgQt7YnO
    # Currently `dap` is down.

    return (
        "https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/data/"
        f"MULTISENSOR_IRCDR/L3S/0.01/v2.00/monthly/{year}/{month:02d}/"
        f"ESACCI-LST-L3S-LST-IRCDR_-0.01deg_1MONTHLY_DAY-{year}{month:02d}01000000-fv2.00.nc"
        # Must add `#mode=bytes` to the end.
        # See https://github.com/Unidata/netcdf4-python/issues/1043
        "#mode=bytes"
    )

    # return pooch.retrieve(lst_data_url, None)

Read land surface temperatures for a specific date using `xarray`:


In [ ]:
minx, miny, maxx, maxy = admin2_gdf.total_bounds

with xr.open_dataset(global_lst_url(year=2020, month=11)) as ds:
    africa_ds = (
        ds.drop_dims("channel")
        .sel(
            lon=slice(minx, maxx),
            lat=slice(miny, maxy),
        )
        .squeeze(drop=True)
        .rio.write_crs("EPSG:4326")
    )

africa_ds

Extract the `lst` data variable, convert from Kelvin to Celsius, and plot:


In [ ]:
zones_da = typing.cast(
    xr.DataArray,
    regionmask.mask_geopandas(
        admin2_gdf,
        africa_ds.lon,
        africa_ds.lat,
    ),
)

zones_da

In [ ]:
len(np.unique(zones_da.values[~np.isnan(zones_da.values)]))

In [ ]:
zones_da.plot()  # type: ignore

Compute zonal means for LST:


In [ ]:
africa_lst_celsius_da = africa_ds.lst - 273.15
# africa_lst_celsius_da.plot(cmap="coolwarm")
africa_lst_celsius_da

In [ ]:
mean_lst_df: pd.DataFrame = (
    xrspatial.zonal.stats(
        zones_da,
        africa_lst_celsius_da,
        stats_funcs=["mean"],
        nodata_values=-1,
    )
    .set_index("zone")  # type: ignore
    .rename(columns={"mean": "mean_lst"})  # type: ignore
)

mean_lst_df

Join the means to the geometries and plot the zonal means. Note that for some
reason, there are fewer zones than geometries (`admin2_gdf`), so we had to set
`"zone"` as the index on the means DataFrame so the values are correctly
aligned:


In [ ]:
mean_lst_gdf = admin2_gdf.join(mean_lst_df, how="inner").dropna()
mean_lst_gdf

In [ ]:
mean_lst_gdf.plot("mean_lst", cmap="coolwarm", legend=True)  # type: ignore

Overlaying our district boundaries over our mean monthly LST data, we can identify (visually) those districts where data is missing.


In [ ]:
ax = admin2_gdf.boundary.plot(alpha=0.15, color="black")
mean_lst_gdf.plot("mean_lst", ax=ax, zorder=-1, cmap="coolwarm", legend=True)